In [1]:
import os
import pandas as pd
import numpy as np
import regex as re
import mytrain_lib as ml
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def plot_results(error,accuracy_train,accuracy_test,confusion_matrix, hyperparams, title=''):

    acc_test_lastepoch = accuracy_test[:,:,-1]  # only interested in last epoch

    # best models of each configuration
    best_cv          = acc_test_lastepoch.argmax(axis=1)
    best_config_cv   = np.unique(acc_test_lastepoch.argmax(axis=0))

    # best configurations are:
    print('config','\t', 'accuracy_test\t', '\taccuracy_train\t', '\terror')

    for c in best_config_cv:
        print(c,'\t', accuracy_test[c,best_cv[c],-1]
                    , accuracy_train[c,best_cv[c],-1]
                    , error[c,best_cv[c],-1])

    if title=='':
        title = datetime.now().strftime("_%m%d_%H%M%S")

    for i,c in enumerate(best_config_cv):
        print(f'Config of {c} - Fold {best_cv[c]}: {hyperparams[c]}')
        ml.dispConfusionMatrix(confusion_matrix[c,best_cv[c]],
                        f'Confusion Matrix: MLP 1x5 {title} - {hyperparams[c]}',
                        f'confmat_{title}_id{i}', save=True)

        

In [4]:
path_train      = 'F://TFG//datasets//data_train//'
path_graphs     = 'F://TFG//graphs//plot_results//'
path_results    = 'F://TFG//results//'

In [5]:
data        = pd.read_csv(path_train+'training_features_DF.csv',sep=';',index_col='wyId')
raw_Data    = pd.read_json('F://TFG//datasets/raw_datasets//RAW_partidos.json').set_index('wyId')

## OBJETIVOS:

- ENTRE LAS DIFERENTES EJECUCIONES    
    - Comparar accuracy
    - Comparar error 
    - Comparar resultados de las diferentes ejecuciones:
        - Comparar optimizador, reduccion de dimensionlidad y hiperparámetros.
    - Concluir que modelos tienen mejores resultados. 
    - Concluir que hiperparámetros afectan más al modelo y deben ser optimizados más detalladamente.
        - Comparar max con la media
<br><br>
- DENTRO DE CADA EJECUCIÓN
    - Evolución del error y accuracy
    - Comprobar los resultados predecidos
    - Concluir si los resultados tienen sentido (p.e. si la red aprende o no)


In [27]:
acc_res, acctrain_res, err_res, hyp_res, cm_res, config = {}, {}, {}, {}, {}, {}

architecture = 'mlp1x5'
path_model = path_results + architecture + '//'
epochs = ['epochs005','epochs020','epochs100']
# epochs = ['anova_mlp5']
ep100 = ''

# reg_exp = re.compile(r'(((sgd)|(adam))(_(\w)+)*(_pca(\d)+)*(_\d\d))+')
reg_exp = re.compile(r'(((sgd)|(adam))(_(\w)+)*(_pca(\d)+)*((_34)|(_09)|(_16)))+')

if architecture!='mlp1x5': epochs = ['']
for ep in epochs:
    paths   = list(filter(reg_exp.match, os.listdir(path_model+ep+'//')))
    for path in paths:
        if ep=='epochs100': ep100 = 'epochs100'
        key = f'{path[:-15]}'
        acc_res[key+ep100]      = np.load(path_model+ep+'//'+path+'//acctest.npy')
        acctrain_res[key+ep100] = np.load(path_model+ep+'//'+path+'//acctrain.npy')
        err_res[key+ep100]      = np.load(path_model+ep+'//'+path+'//error.npy')
        hyp_res[key+ep100]      = np.load(path_model+ep+'//'+path+'//hyperparams.npy',allow_pickle=True)
        cm_res[key+ep100]       = np.load(path_model+ep+'//'+path+'//confmat.npy')
        # config[key+ep100]       = {'layers': int(architecture[-3]),'units':int(architecture[-1]),
        #                             'opt':(lambda x: 'sgd' if 'sgd' in x else 'adam')(key),'epochs': int(ep[6:])}

keys = (list(acc_res.keys()))
np.array(keys)

array(['adam_anova02_epochs5', 'adam_anova05_epochs5',
       'adam_anova07_epochs5', 'adam_anova10_epochs5',
       'adam_anova2_epochs5', 'adam_anova5_epochs5',
       'adam_anova7_epochs5', 'adam_epochs5', 'adam_featsel_epochs5',
       'adam_pca10_epochs5_', 'adam_pca3_epochs5_', 'adam_pca5_epochs5_',
       'sgd_anova02_epochs5', 'sgd_anova05_epochs5',
       'sgd_anova07_epochs5', 'sgd_anova10_epochs5', 'sgd_anova2_epochs5',
       'sgd_anova5_epochs5', 'sgd_anova7_epochs5', 'sgd_epochs5',
       'sgd_featsel_epochs5', 'sgd_pca10_epochs5_', 'sgd_pca3_epochs5_',
       'sgd_pca5_epochs5_', 'adam_anova02_epochs20',
       'adam_anova05_epochs20', 'adam_anova07_epochs20',
       'adam_anova10_epochs20', 'adam_anova2_epochs20',
       'adam_anova5_epochs20', 'adam_anova7_epochs20', 'adam_epochs20',
       'adam_featsel_epochs20', 'adam_pca10_epochs20',
       'adam_pca3_epochs20_', 'adam_pca5_epochs20',
       'sgd_anova02_epochs20', 'sgd_anova05_epochs20',
       'sgd_anova07_epochs

## Multilayer Perceptron: 5 units

In [34]:
acc_cvs = np.mean(acc_res['adam_anova10_epochs100epochs100'][:,:,-1],axis=1)
print('Best accuracy with Adam and ANOVA-7 input and 20 epochs: {:.2f}% '.format(100*np.max(acc_cvs)))
print('Average accuracy with Adam and ANOVA-7 input and 20 epochs: {:.2f}% '.format(100*np.mean(acc_cvs)))

Best accuracy with Adam and ANOVA-7 input and 20 epochs: 60.45% 
Average accuracy with Adam and ANOVA-7 input and 20 epochs: 56.87% 


In [33]:
np.max(acc_res['adam_anova10_epochs100epochs100'][:,:,-1])

0.6227272727272727

In [ ]:
acc_res['sgd_pca10_epochs20'].shape

In [32]:
for k in keys:
    if 'anova10' in k:
        acc_cvs = np.mean(acc_res[k][:,:,-1],axis=1)
        print('{} ------- {:.2f}% '.format(k,100*np.max(acc_cvs)))
        # print('Average accuracy {}: {:.2f}% '.format(k,100*np.mean(acc_cvs)))
        print()

adam_anova10_epochs5 ------- 59.82% 

sgd_anova10_epochs5 ------- 57.91% 

adam_anova10_epochs20 ------- 59.55% 

sgd_anova10_epochs20 ------- 59.36% 

adam_anova10_epochs100epochs100 ------- 60.45% 

adam_anova10_epochs100 ------- 56.45% 

sgd_anova10_epochs100epochs100 ------- 59.27% 



### Comparison of optimizers and dimensional reduction

In [ ]:
xticks   = [k[:-8] for k in keys[:12]]
xticks

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1,figsize=(13,10),constrained_layout = True)

fig.suptitle('Accuracy on experiments',fontsize=18)

idx_lim = 6
w = 0.25
labels = ['5', '20', '100']

# ADAM
ax1.set_title('Adam models')

colors = ['#4EE043','#F3C54F','#64F0F9']
for i,ep in enumerate(['_epochs5','_epochs20','_']):
    max_values  = [ np.max(acc_res[k+ep][:,:,-1]) for k in xticks[:idx_lim]]
    mean_values = [np.mean(acc_res[k+ep][:,:,-1]) for k in xticks[:idx_lim]]
    ax1.bar(x=np.arange(idx_lim)+w*i,width=w,height=max_values,
                color=colors[i],label=labels[i])

ax1.twinx()

colors = ['#3CAD34','#B3913B','#5AD7E0']
for i,ep in enumerate(['_epochs5','_epochs20','_']):
    max_values  = [ np.max(acc_res[k+ep][:,:,-1]) for k in xticks[:idx_lim]]
    mean_values = [np.mean(acc_res[k+ep][:,:,-1]) for k in xticks[:idx_lim]]
    ax1.bar(x=np.arange(idx_lim)+w*i,width=w,height=mean_values,
                color=colors[i])

ax1.set_ylabel('accuracy')
ax1.set_ylim([0,1])
ax1.legend(title='Epochs:')
ax1.grid()
ax1.set_xticks(ticks=np.arange(6)+w)
ax1.set_xticklabels(xticks[:idx_lim],fontsize=12,rotation=25)

# SGD
ax2.set_title('Stochastic Gradient Descend models')

colors = ['#4EE043','#F3C54F','#64F0F9']
for i,ep in enumerate(['_epochs5','_epochs20','_']):
    max_values  = [ np.max(acc_res[k+ep][:,:,-1]) for k in xticks[idx_lim:]]
    mean_values = [np.mean(acc_res[k+ep][:,:,-1]) for k in xticks[idx_lim:]]
    ax2.bar(x=np.arange(idx_lim)+w*i,width=w,height=max_values,
                color=colors[i],label=labels[i])

ax2.twinx()

colors = ['#3CAD34','#B3913B','#5AD7E0']
for i,ep in enumerate(['_epochs5','_epochs20','_']):
    max_values  = [ np.max(acc_res[k+ep][:,:,-1]) for k in xticks[idx_lim:]]
    mean_values = [np.mean(acc_res[k+ep][:,:,-1]) for k in xticks[idx_lim:]]
    ax2.bar(x=np.arange(idx_lim)+w*i,width=w,height=mean_values,
                color=colors[i])

ax2.set_ylabel('accuracy')
ax2.set_ylim([0,1])
# ax2.legend(title='Epochs:')
ax2.grid()
ax2.set_xticks(ticks=np.arange(6)+w)
ax2.set_xticklabels(xticks[idx_lim:],fontsize=12,rotation=25)

plt.savefig(path_graphs + 'accuracy_MLP1x5_epochs.jpg', format='jpg', dpi=200)
plt.show()

In [ ]:
# ERROR

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(13,10),constrained_layout = True)

fig.suptitle('Accuracy on experiments',fontsize=18)

idx_lim = 6
w = 0.25
labels = ['5', '20', '100']
colors = ['#FF3A1C','#C42D16','#BF5D4E']

# ADAM
ax1.set_title('Adam models')

for i,ep in enumerate(['_epochs5','_epochs20','_']):
    max_values  = [ np.min(acc_res[k+ep][:,:,-1]) for k in xticks[:idx_lim]]
    ax1.bar(x=np.arange(idx_lim)+w*i,width=w,height=max_values,
                color=colors[i],label=labels[i])

ax1.set_ylabel('error')
ax1.set_ylim([0,0.5])
ax1.legend(title='Epochs:')
ax1.set_xticks(ticks=np.arange(6)+w)
ax1.set_xticklabels(xticks[:idx_lim],fontsize=12,rotation=25)

# SGD
ax2.set_title('Stochastic Gradient Descend models')

for i,ep in enumerate(['_epochs5','_epochs20','_']):
    max_values  = [ np.min(acc_res[k+ep][:,:,-1]) for k in xticks[idx_lim:]]
    ax2.bar(x=np.arange(idx_lim)+w*i,width=w,height=max_values,
                color=colors[i],label=labels[i])

ax2.set_ylabel('error')
ax2.set_ylim([0,0.5])
ax2.set_xticks(ticks=np.arange(6)+w)
ax2.set_xticklabels(xticks[idx_lim:],fontsize=12,rotation=25)

plt.savefig(path_graphs + 'error_MLP1x5_epochs.jpg', format='jpg', dpi=200)
plt.show()

In [ ]:
max_values  = [ np.max(acc_res[k][:,:,-1]) for k in keys]
mean_values = [np.mean(acc_res[k][:,:,-1]) for k in keys]

color = ['#1183C4','#17CA45','#0E5F8E','#108C30']

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(11,10),constrained_layout = True)

fig.suptitle('Accuracy on experiments',fontsize=18)

idx_lim = 13

ax1.set_title('Adam models')
ax1.bar(x=keys[:idx_lim],height=max_values[:idx_lim],color='#17CA45',label='max value')
ax1.twinx()
ax1.bar(x=keys[:idx_lim],height=mean_values[:idx_lim],color='#108C30',label='mean value')
ax1.set_ylabel('accuracy')
ax1.set_ylim([0,1])
ax1.legend()
ax1.set_xticklabels(keys[:idx_lim],fontsize=9,rotation=25)
# ax1.grid()

ax2.set_title('SGD models')
ax2.bar(x=keys[idx_lim:],height=max_values[idx_lim:],color='#1183C4',label='max value')
ax2.twinx()
ax2.bar(x=keys[idx_lim:],height=mean_values[idx_lim:],color='#0E5F8E',label='mean value')
ax2.set_xlabel('experiments'); ax2.set_ylabel('accuracy')
ax2.set_xticklabels(ax2.get_xticks(), rotation = 0)
ax2.legend()
ax2.set_ylim([0,1])
ax2.set_xticklabels(keys[idx_lim:],fontsize=9,rotation=25)
# ax2.grid()

plt.savefig(path_graphs + 'accuracy_MLP1x5.jpg', format='jpg', dpi=200)
plt.show()

In [ ]:
max_values  = [ np.min(err_res[k][:,:,-1]) for k in keys]

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(11,10),constrained_layout = True)

fig.suptitle('Error on experiments',fontsize=18)

idx_lim = 13

ax1.set_title('Adam models')
ax1.bar(x=keys[:idx_lim],height=max_values[:idx_lim],color='#FF1100')
ax1.set_ylabel('error')
ax1.set_ylim([0,max(max_values)+0.2])
ax1.legend()
ax1.set_xticklabels(keys[:idx_lim],fontsize=9,rotation=25)
# ax1.grid()

ax2.set_title('SGD models')
ax2.bar(x=keys[idx_lim:],height=max_values[idx_lim:],color='#FF4300')
ax2.set_xlabel('experiments'); ax2.set_ylabel('error')
ax2.set_xticklabels(ax2.get_xticks(), rotation = 0)
ax2.legend()
ax2.set_ylim([0,max(max_values)+0.2])
ax2.set_xticklabels(keys[idx_lim:],fontsize=9,rotation=25)
# ax2.grid()

plt.savefig(path_graphs + 'error_MLP1x5.jpg', format='jpg', dpi=200)
plt.show()

### Plot de las mejores ejecuciones

Mostramos la evolucion del error durante el entrenamiento.

In [ ]:
import importlib
importlib.reload(ml)

In [ ]:
np.array(keys)

In [ ]:
# plot results

k = 'sgd_varthr14_'
print(k)
print()

plot_results(err_res[k],acctrain_res[k],acc_res[k],cm_res[k],hyp_res[k],k)

In [ ]:
toplot = []
for r,c in zip([41,255,16,241,30],[0,3,4,3,4]):
    toplot.append(err_res['adam'][r,c])

plot_data = np.array(toplot)

In [ ]:
plt.figure(figsize=(15,8))

color  = ['#FA5C7A','#564FD6','#64EDC2','#CED64F','#FA9D4E']
label  = [41,255,16,241,30]

for i,p in enumerate(plot_data):
    plt.plot(p,color=color[i],label=label[i])

plt.title(f'Error Adam')
plt.xticks(np.arange(0,101,10))
plt.legend(title='Config.')
# plt.grid()
plt.xlabel('epochs')
plt.ylabel('error')
plt.ylim([0.2,np.max(plot_data)+np.min(plot_data)-0.2])
plt.savefig(path_graphs + f'error_adam.jpg', format='jpg', dpi=200)

plt.show()

### Box Plot de cada hiperparámetro

Mostraremos unas gráficos de box plot para analizar la influencia de cada hiperparámetro entrenado en el porcentaje de acierto de nuestros modelos.

In [ ]:
keys

#### Comparacion hiperparámetros (Batch-size, learning rate, scaler...)

In [ ]:
acc_array = np.concatenate([acc_res[k][:,:,-1] for k in keys]).reshape(-1)

fig, (ax1,ax3,ax4) = plt.subplots(1,3,figsize=(24,6))
fig.suptitle('General Hyperparameters Box Plot',fontsize=18)

# scalers
lr_array  = np.concatenate([hyp_res[k][:,0] for k in keys])
lr_array  = np.array([[type(lr),type(lr),type(lr),type(lr),type(lr)] for lr in lr_array]).reshape(-1)
assert len(lr_array) == len(acc_array)
idxs = [np.where(lr_array == lr) for lr in list(set(lr_array))]
plot_data = [acc_array[i] for i in idxs]
ax1.boxplot(plot_data)
ax1.set_ylabel('Accuracy')
ax1.set_title('Scalers Box Plot')
ax1.set_xlabel('Scaler')
ax1.set_xticklabels(['MinMax','None Scaler','Normaliz.'])

# learning-rate
lr_array  = np.concatenate([hyp_res[k][:,2] for k in keys])
lr_array  = np.array([np.ones(5)*lr for lr in lr_array]).reshape(-1)
assert len(lr_array) == len(acc_array)
idxs = [np.where(lr_array == lr) for lr in list(set(lr_array))]
plot_data = [acc_array[i] for i in idxs]
ax3.boxplot(plot_data)
ax3.set_title('Learning rate Box Plot')
ax3.set_xlabel('Learning rate')
ax3.set_xticklabels(list(set(lr_array)))

# batch-size
lr_array  = np.concatenate([hyp_res[k][:,-1] for k in keys])
lr_array  = np.array([np.ones(5)*lr for lr in lr_array]).reshape(-1)
assert len(lr_array) == len(acc_array)
idxs = [np.where(lr_array == lr) for lr in list(set(lr_array))]
plot_data = [acc_array[i] for i in idxs]
ax4.boxplot(plot_data)
ax4.set_title('Batch-size Box Plot')
ax4.set_xlabel('Batch size')
ax4.set_xticklabels(list(set(lr_array)))

plt.savefig(path_graphs + f'bxplt_generalhyper.jpg', format='jpg', dpi=200)
plt.show()

#### Comparacion optimizador (SGD vs Adam)

In [ ]:
plot_data = ([np.concatenate([acc_res[k][:,:,-1] for k in keys[:4]]).reshape(-1), 
                np.concatenate([acc_res[k][:,:,-1] for k in keys[4:]]).reshape(-1)])

print(np.array(plot_data).shape)
plt.figure(figsize=(4,6))
plt.boxplot(plot_data)
plt.title('Optimizer Boxplot',fontsize=15)
plt.ylabel('Accuracy')
plt.xlabel('Optimizer')
# plt.grid()
plt.xticks(ticks=[1,2],labels=['Adam','Stochastic Gradient Descend'])
# plt.savefig(path_graphs + f'bxplt_optimizer.jpg', format='jpg', dpi=200)
plt.show()

#### Comparacion de Momentum y Nesterov (para SGD)

In [ ]:
acc_array = np.concatenate([acc_res[k][:,:,-1] for k in keys[:4]]).reshape(-1)

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(13,6))
fig.suptitle('SGD Hyperparameters Box Plot',fontsize=18)

# momentum
lr_array  = np.concatenate([hyp_res[k][:,3] for k in keys[:4]])
lr_array  = np.array([np.ones(5)*lr for lr in lr_array]).reshape(-1)
assert len(lr_array) == len(acc_array)
idxs = [np.where(lr_array == lr) for lr in list(set(lr_array))]
plot_data = [acc_array[i] for i in idxs]
print(np.array(plot_data).shape)
ax1.boxplot(plot_data)
ax1.set_title('Momentum Box Plot')
ax1.set_xlabel('Momentum')
print(type(ax1))
ax1.set_xticklabels(list(set(lr_array)))


# nesterov
lr_array  = np.concatenate([hyp_res[k][:,4] for k in keys[:4]])
lr_array  = np.array([np.ones(5)*lr for lr in lr_array]).reshape(-1)
assert len(lr_array) == len(acc_array)
idxs = [np.where(lr_array == lr) for lr in list(set(lr_array))]
plot_data = [acc_array[i] for i in idxs]
ax2.boxplot(plot_data)
ax2.set_title('Nesterov Box Plot')
ax2.set_xlabel('Nesterov')
ax2.set_xticklabels(list(set(lr_array)))

plt.ylabel('Accuracy')
# plt.savefig(path_graphs + f'bxplt_sgdhyp.jpg', format='jpg', dpi=200)
plt.show()

#### Comparacion hyperparámetros Adam

In [ ]:
acc_array = np.concatenate([acc_res[k][:,:,-1] for k in keys[:4]]).reshape(-1)

fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(18,6))
fig.suptitle('Adam Hyperparameters Box Plot',fontsize=18)

# b1
lr_array  = np.concatenate([hyp_res[k][:,3] for k in keys[:4]])
lr_array  = np.array([np.ones(5)*lr for lr in lr_array]).reshape(-1)
assert len(lr_array) == len(acc_array)
idxs = [np.where(lr_array == lr) for lr in list(set(lr_array))]
plot_data = [acc_array[i] for i in idxs]
ax1.boxplot(plot_data)
ax1.set_ylabel('Accuracy')
ax1.set_title('b1 Box Plot')
ax1.set_xlabel('b1')
print(type(ax1))
ax1.set_xticklabels(list(set(lr_array)))


# b2
lr_array  = np.concatenate([hyp_res[k][:,4] for k in keys[:4]])
lr_array  = np.array([np.ones(5)*lr for lr in lr_array]).reshape(-1)
assert len(lr_array) == len(acc_array)
idxs = [np.where(lr_array == lr) for lr in list(set(lr_array))]
plot_data = [acc_array[i] for i in idxs]
print(np.array(plot_data).shape)
ax2.boxplot(plot_data)
ax2.set_title('b2 Box Plot')
ax2.set_xlabel('b2')
ax2.set_xticklabels(list(set(lr_array)))

# weight_decay
lr_array  = np.concatenate([hyp_res[k][:,5] for k in keys[:4]])
lr_array  = np.array([np.ones(5)*lr for lr in lr_array]).reshape(-1)
assert len(lr_array) == len(acc_array)
idxs = [np.where(lr_array == lr) for lr in list(set(lr_array))]
plot_data = [acc_array[i] for i in idxs]
ax3.boxplot(plot_data)
ax3.set_title('Weight decay Box Plot')
ax3.set_xlabel('Weight decay')
ax3.set_xticklabels(list(set(lr_array)))

# plt.savefig(path_graphs + f'bxplt_adamhyp.jpg', format='jpg', dpi=200)
plt.show()

#### Comparacion de PCAs

In [ ]:
keys

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(14,6))
fig.suptitle('PCA Box Plot',fontsize=18)

# SGD
plot_data = np.array([acc_res[i][:,:,-1] for i in keys[4:]]).reshape(4,-1)
ax1.boxplot(list(plot_data))
ax1.set_ylabel('Accuracy')
ax1.set_title('PCAs with SGD Box Plot')
# ax1.set_xlabel('PCA')
ax1.set_xticklabels(['No Dim.Reduction','PCA 3','PCA 5','PCA 10'])

# ADAM
plot_data = np.array([acc_res[i][:,:,-1] for i in keys[:4]]).reshape(4,-1)
ax2.boxplot(list(plot_data))
ax2.set_ylabel('Accuracy')
ax2.set_title('PCAs with Adam Box Plot')
# ax2.set_xlabel('PCA')
ax2.set_xticklabels(['No Dim.Reduction','PCA 3','PCA 5','PCA 10'])

plt.savefig(path_graphs + f'bxplt_sgdvsadam.jpg', format='jpg', dpi=200)
plt.show()

### Comparacion Batch-Normalization

Sobre Batch-Normalization:

https://machinelearningmastery.com/batch-normalization-for-training-of-deep-neural-networks/

In [ ]:
max_values  = [ np.max(acc_res[k][:,:,-1]) for k in np.array(keys)[[4,5,6]]]
mean_values = [np.mean(acc_res[k][:,:,-1]) for k in np.array(keys)[[4,5,6]]]

# color = ['#D14444','#B5189D','#6F2424','#630E56']
# color = ['#1183C4','#17CA45','#0E5F8E','#108C30']

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,6))

fig.suptitle('Accuracy on experiments based on Batch-normalization',fontsize=18)
ax1.set_title('Max Accuracy')
ax1.bar(x=list(np.array(keys)[[4,5,6]]),height=max_values,color='#D14444')
ax1.set_xlabel('experiments'); ax1.set_ylabel('accuracy')
ax1.set_ylim([0,1])
ax1.set_xticklabels(['BN 1', 'BN 2', 'No BN'],fontsize=8)

ax2.set_title('Mean Accuracy')
ax2.bar(x=list(np.array(keys)[[4,5,6]]),height=mean_values,color='#6F2424')
ax2.set_xticklabels(ax2.get_xticks(), rotation = 0)
ax2.set_ylim([0,1])
ax2.set_xticklabels(['BN 1', 'BN 2', 'No BN'],fontsize=8)

plt.savefig(path_graphs + 'accuracy_expBatchNorm.jpg', format='jpg', dpi=200)
plt.show()

In [ ]:
plot_data = [acc_res[k][:,:,-1].reshape(-1) for k in np.array(keys)[[4,5,6]]]
plt.figure(figsize=(9,7))
plt.boxplot(plot_data)
plt.ylabel('Accuracy')
plt.xlabel('Batch-normalization')
plt.xticks(ticks=[1,2,3],labels=['BN 1', 'BN 2', 'No BN'])
plt.title('Batch-Normalization Box Plot')
plt.savefig(path_graphs + f'bxplt_batchnorm.jpg', format='jpg', dpi=200)
plt.show()

### Comparacion Feature selection

### Cuales son los mejores modelos para predecir empates? O para predecir que gana el visitante?

Podrá ser útil en un futuro, cuando intentemos hacer un modelo basándonos en la técnica de bagging con varias redes entrenadas.

## Multilayer Perceptron: 3 units

### Comparacion of optimizers and dimensional reduction

## Depuracion entrenamiento Red Neuronal

Vamos a depurar el entrenamiento de nuestro modelo para intentar identificar las causas por las cuales no aprende correctamente.